In [ ]:
# default_exp mgmnt.prep.bpe

# BPE Preprocessing

> This module comprises all preprocessing techniques applied to software artifacts:
>
>> Text-based Artifacts: Classical preprocessing (stemming, lemas, etc) and BPE Binary Artifacts:
>
>> To Do Vision-based Artifacts:
>
>> To Do Parsing: Techniques to control and manipulate source code (complete with deep generator project)

In [ ]:
! pip install dit

In [ ]:
! pip install fastprogress

You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
# export
# Imports
import pandas as pd
import random
import sentencepiece as sp

from fastprogress.fastprogress import master_bar
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
def jsonl_list_to_dataframe(file_list, columns=None):
    """Load a list of jsonl.gz files into a pandas DataFrame."""
    return pd.concat([pd.read_json(f,
                                   orient='records', 
                                   compression='gzip',
                                   lines=True)[columns] 
                      for f in file_list], sort=False)

In [ ]:
# export
def get_dfs(path):
    """
        Grabs the different data splits and converts them into dataframes.
        Expects format from Code Search Net Challenge.
    """
    dfs = []
    for split in ["train", "valid", "test"]:
        files = sorted((path/split).glob("**/*.gz"))
        df = jsonl_list_to_dataframe(files, ["code", "docstring"])
        dfs.append(df)
        
    return dfs

In [ ]:
path = Path('/tf/data/')

In [ ]:
df_trn, df_val, df_tst = get_dfs(path/"java/final/jsonl")
df_trn.head()

,code,docstring
0,protected final void bindIndexed(Configuration...,Bind indexed elements to the supplied collecti...
1,public void setServletRegistrationBeans(\n\t\t...,Set {@link ServletRegistrationBean}s that the ...
2,public void addServletRegistrationBeans(\n\t\t...,Add {@link ServletRegistrationBean}s for the f...
3,public void setServletNames(Collection<String>...,Set servlet names that the filter will be regi...
4,public void addServletNames(String... servletN...,Add servlet names for the filter.\n@param serv...


In [ ]:
# Save some test data
df_trn.sample(frac = 0.01).to_csv('./test_data/trn.csv', index = False)
df_val.sample(frac = 0.01).to_csv('./test_data/val.csv', index = False)
df_tst.sample(frac = 0.01).to_csv('./test_data/tst.csv', index = False)

In [ ]:
# export
def df_to_txt_file(df, output, cols):
    """Converts a dataframe and converts it into a text file that SentencePiece can use to train a BPE model"""
    if cols is None: cols = list(df.columns)
    merged_df = pd.concat([df[col] for col in cols])
    
    with open(output/'text.txt', 'w') as f:
        f.write('\n'.join(list(merged_df)))
    return output/'text.txt'

In [ ]:
# export
def sp_model_from_df(df, output, model_name, cols = None):
    """Trains a SentencePiece BPE model from a pandas dataframe"""
    fname = df_to_txt_file(df, output, cols)
    sp.SentencePieceTrainer.train(f'--input={fname} --model_prefix={output / model_name} --hard_vocab_limit=false')

In [ ]:
# export
def sp_model_from_glob(path, glob, model_name):
    fns = list(path.glob(glob))
    fns = ",".join(map(str, fns))
    sp.SentencePieceTrainer.train(f'--input={fns} --model_prefix={path / model_name} --hard_vocab_limit=false')

In [ ]:
# export
def gen_hugface_model(df, output, tokenizer = ByteLevelBPETokenizer(), vocab_sz = 30_000, min_freq = 3, cols = None):
    fname = df_to_txt_file(df, output, cols)
    tokenizer.train(files = [str(fname)], vocab_size = vocab_sz, min_frequency = min_freq, special_tokens=[
        "<s>",
        "<pad>",
        "</s>",
        "<unk>",
        "<mask>",
    ])
    
    return tokenizer

In [ ]:
path = Path("./test_data")
model_name = "test"

In [ ]:
df = pd.read_csv(path / 'trn.csv')
df.head()

,code,docstring
0,private static void createCode(String packageN...,Create the Java
1,@Override\n public void flushCache() {\n ...,LI3492-2
2,"public void addRule(IntDependency dependency, ...",Add this dependency with the given count to th...
3,@Override\n public boolean removeIfEquals(K k...,Remove the object from the cache.
4,public void marshall(DatasetContentDeliveryDes...,Marshall the given parameter object.


In [ ]:
tokenizer = gen_hugface_model(df, path)

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)

In [ ]:
print(tokenizer.encode("public static void main(String[] args) { getDirFromLib(); }").tokens)

['<s>', 'public', 'Ġstatic', 'Ġvoid', 'Ġmain', '(', 'String', '[]', 'Ġargs', ')', 'Ġ{', 'Ġget', 'Dir', 'From', 'Lib', '();', 'Ġ}', '</s>']


In [ ]:
tokenizer.save(str(path), "java_tokenizer")

['test_data/java_tokenizer-vocab.json', 'test_data/java_tokenizer-merges.txt']

In [ ]:
dummy_data = {
        'first': ['1', '2', '6', '7', '8'],
        'second': ['K', 'M', 'O', 'Q', 'S'],
        'third': ['L', 'N', 'P', 'R', 'T']}

In [ ]:
df = pd.DataFrame(dummy_data2); df

,id,Feature1,Feature2
0,1,K,L
1,2,M,N
2,6,O,P
3,7,Q,R
4,8,S,T


In [ ]:
df_to_txt_file(df, Path('./test_data'), list(df.columns))

PosixPath('test_data/text.txt')

In [ ]:
path = Path("./test_data")
model_name = "test"

In [ ]:
sp_model_from_dfs(df, path, model_name, list(df.columns))

In [ ]:
spm = sp.SentencePieceProcessor()
spm.Load(str(path/f"{model_name}.model"))

True

In [ ]:
spm.EncodeAsPieces("Hello, world!")

['▁', 'Hello,', '▁', 'world!']

In [ ]:
# export
def tokenize_fns(fns, tokenizer, exts, output, data_type):
    docs = []
    for fn in fns:
        system = fn.parent.name
        output_path = output/system/data_type
        output_path.mkdir(parents=True, exist_ok=True)
        files = []
        for ext in exts:
            files.extend(fn.glob(f'**/*.{ext}'))
        for file in files:
            if 'README' not in file.name:
                with open(file, encoding='ISO-8859-1') as f:
                    docs.append(tokenizer.EncodeAsPieces(f.read()))
                with open((output_path/file.name).with_suffix('.bpe'), 'w') as f:
                    f.write(' '.join(docs[-1]))
            
    return docs

In [ ]:
# export
def read_bpe_files(path):
    bpe_files = []
    for file in path.glob('**/*.bpe'):
        with open(file) as f:
            bpe_files.append(f.read().split(' '))
    
    return bpe_files

In [ ]:
# export
def split_lines_to_files(lines, fn_pattern, output_path, tokenizer):
    for line in lines:
        fn, content = line.split(fn_pattern)
        fn = fn.replace('"', '')
        fn = fn.replace(' Test ', '')
        content = tokenizer.EncodeAsPieces(content)
        with open((output_path/fn).with_suffix('.bpe'), 'w') as f:
                    f.write(' '.join(content))

In [ ]:
path = Path('../benchmarking/traceability/')

In [ ]:
spm = sp.SentencePieceProcessor()
spm.Load(str(path/'datasets/italian/italian_bpe.model'))

True

In [ ]:
ebt_path = path/'datasets/italian/ebt'; ebt_path

PosixPath('../benchmarking/traceability/datasets/italian/ebt')

In [ ]:
with open(ebt_path/'[ebt-raw-req].txt') as f:
    split_lines_to_files(f.read().split('\n')[:-1], '\t', path/'testbeds/bpe/italian/ebt/req', spm)

In [ ]:
with open(ebt_path/'[ebt-raw-tc].txt') as f:
    split_lines_to_files(f.read().split('\n')[:-1], 'case:', path/'testbeds/bpe/italian/ebt/tc', spm)

In [ ]:
# export
def get_ground_truth(path, language):
    all_links = pd.DataFrame([], columns = [
        'sys', 'from_type', 'to_type', 'from_file', 'to_file', 'from_doc', 'to_doc'
    ])
    for fn in path.glob('*.txt'):
        content = str(fn.name).split('.')[0][1:-1]
        content = content.split('-')
        
        sys, from_type, to_type = content[0], content[2], content[4]
        
        with open(fn) as f:
            links = f.read().split('\n')[:-1]
            
        for link in links:
            link = link.split(' ')
            root, children = link[0], link[1:]
            root = Path(root).with_suffix('.bpe').name
            with open(path.parent.parent/'bpe'/language/sys/from_type/root) as f:
                root_content = f.read().split(' ')
            children = [Path(child).with_suffix('.bpe').name for child in children]
            children = [Path('.'.join(str(child).split('.')[-2:])) for child in children]
            for child in children:
                with open(path.parent.parent/'bpe'/language/sys/to_type/child) as f:
                    child_content = f.read().split(' ')
                all_links = all_links.append({'sys': sys,
                                              'from_type': from_type,
                                              'to_type': to_type,
                                              'from_file': root,
                                              'to_file': str(child),
                                              'from_doc': root_content,
                                              'to_doc': child_content},
                                             ignore_index=True)
            
    return all_links

In [ ]:
# export
def get_non_ground_truth(path, language, gt):
    all_non_links = []
    
    existing_links = ['->'.join(link) for link in zip(gt['from_file'].to_list(), gt['to_file'].to_list())]
    bpe_files = list(path.glob('**/*.bpe'))
    random.shuffle(bpe_files)
    for i in bpe_files[:500]:
        sys = i.parent.parent.name
        from_type = i.parent.name
        if str(from_type) != 'req': continue
        with open(i) as f:
            i_content = f.read().split(' ')
        random.shuffle(bpe_files)
        for j in bpe_files[:500]:
            if i == j: continue
            if '->'.join([i.name, j.name]) in existing_links: continue
            to_type = j.parent.name
            if str(to_type) == 'req': continue
#             if from_type == to_type: continue
            with open(j) as f:
                j_content = f.read().split(' ')
            all_non_links.append([sys, from_type, to_type, i.name, j.name, i_content, j_content])
    
    all_non_links = pd.DataFrame(all_non_links, columns = [
        'sys', 'from_type', 'to_type', 'from_file', 'to_file', 'from_doc', 'to_doc'
    ])
    return all_non_links

In [ ]:
# export
def gen_gt_ngt(path, lang):
    gt = get_ground_truth(path/'groundtruth'/lang, lang)
    ngt = get_non_ground_truth(path/'bpe'/lang, lang, gt)
    
    return gt, ngt

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_mgmnt.prep.i.ipynb.
Converted 01_exp.i.ipynb.
Converted 02_mgmnt.db.mongo.ipynb.
Converted 03_repr.i.ipynb.
Converted 04_mining.ir.model.ipynb.
Converted 05_mining.ir.i.ipynb.
Converted 06_benchmark.traceability.ipynb.
Converted 07_repr.roberta.train.ipynb.
Converted 08_exp.info.ipynb.
Converted 09_desc.stats.ipynb.
Converted 10_vis.ipynb.
Converted 11_mgmnt.prep.nltk.ipynb.
Converted 12_repr.roberta.eval.ipynb.
Converted 14_mgmnt.prep.bpe.ipynb.
Converted 15_desc.metrics.se.ipynb.
Converted 16_repr.word2vec.train.ipynb.
Converted 17_repr.doc2vec.train.ipynb.
Converted 18_repr.doc2vec.eval.ipynb.
Converted 19_repr.word2vec.eval.ipynb.
Converted 20_benchmark.codegen.ipynb.
Converted 21_inf.i.ipynb.
Converted 22_inf.bayesian.ipynb.
Converted 23_inf.causal.ipynb.
Converted aa_blog.example.ipynb.
Converted ab_templates.example.ipynb.
Converted ac_emp.eval.pp1.rq1.ipynb.
Converted ad_emp.eval.pp1.rq2.ipynb.
Converted ae_emp.eval.pp1.rq3.ipynb.
Converted af_emp.eval.pp1.rq4.ipyn